In [11]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D
import os
import numpy as np
import cv2
from tqdm import tqdm
import random
tf.__version__

'2.0.0'

In [12]:
NUM_CLASSES = 3
BATCH_SIZE = 64
TRAIN_DIR = "./FIRE-SMOKE-DATASET/Train"
TEST_DIR = "./FIRE-SMOKE-DATASET/Test"
CATEGORIES = ['Fire', 'Neutral']
image_size = 64

In [16]:
def create_data(DIR):
    training_data = []
    for category in CATEGORIES:  

        path = os.path.join(DIR,category) 
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=C 1=O

        for img in tqdm(os.listdir(path)):  # iterate over each image
            try:
                img_array = cv2.imread(os.path.join(path,img))  # convert to array
                new_array = cv2.resize(img_array, (image_size, image_size))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
              
    return training_data

training_data = create_data(TRAIN_DIR)
test_data = create_data(TEST_DIR)
random.shuffle(training_data)

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 305.67it/s]


In [18]:
def process_data(data):
    X = []
    Y = []
    n = 0

    for features,label in data:
        X.append(features)
        Y.append(label)
        n += 1

    X = np.array(X).reshape(-1, image_size, image_size, 3)
    Y = np.array(Y)
    
    return X, Y, n

X_train, Y_train, total_train = process_data(training_data)
X_test, Y_test, total_val = process_data(test_data)

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_image_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.5
    )

validation_image_generator = ImageDataGenerator(rescale=1./255)

train_data_gen = train_image_generator.flow(X_train, Y_train, batch_size = BATCH_SIZE)

val_data_gen = validation_image_generator.flow(X_test, Y_test, batch_size = BATCH_SIZE)

In [20]:
model = Sequential()

model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(image_size,image_size,3)))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=128, activation='relu'))

model.add(Dense(units=2, activation = 'softmax'))

In [21]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='model_{epoch}.h5',
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // BATCH_SIZE,
    epochs=100,
    validation_data=val_data_gen,
    validation_steps=total_val // BATCH_SIZE,
    callbacks = callbacks
)

Epoch 1/100
27/28 [===========================>..] - ETA: 58s - loss: 0.6892 - acc: 0.48 - ETA: 30s - loss: 0.6695 - acc: 0.50 - ETA: 20s - loss: 0.6546 - acc: 0.58 - ETA: 15s - loss: 0.7136 - acc: 0.55 - ETA: 11s - loss: 0.7081 - acc: 0.53 - ETA: 9s - loss: 0.7034 - acc: 0.5474 - ETA: 8s - loss: 0.7002 - acc: 0.547 - ETA: 7s - loss: 0.6996 - acc: 0.547 - ETA: 6s - loss: 0.6944 - acc: 0.551 - ETA: 5s - loss: 0.6922 - acc: 0.548 - ETA: 4s - loss: 0.6902 - acc: 0.550 - ETA: 4s - loss: 0.6861 - acc: 0.555 - ETA: 3s - loss: 0.6816 - acc: 0.574 - ETA: 3s - loss: 0.6810 - acc: 0.575 - ETA: 2s - loss: 0.6734 - acc: 0.584 - ETA: 2s - loss: 0.6751 - acc: 0.579 - ETA: 2s - loss: 0.6695 - acc: 0.586 - ETA: 1s - loss: 0.6598 - acc: 0.597 - ETA: 1s - loss: 0.6549 - acc: 0.604 - ETA: 1s - loss: 0.6473 - acc: 0.615 - ETA: 1s - loss: 0.6472 - acc: 0.620 - ETA: 1s - loss: 0.6378 - acc: 0.629 - ETA: 0s - loss: 0.6352 - acc: 0.632 - ETA: 0s - loss: 0.6338 - acc: 0.634 - ETA: 0s - loss: 0.6313 - acc: 0.63

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\yemq3\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-21-a565c6a83c39>", line 21, in <module>
    callbacks = callbacks
  File "c:\users\yemq3\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\keras\engine\training.py", line 1297, in fit_generator
    steps_name='steps_per_epoch')
  File "c:\users\yemq3\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\keras\engine\training_generator.py", line 221, in model_iteration
    batch_data = _get_next_batch(generator)
  File "c:\users\yemq3\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\keras\engine\training_generator.py", line 363, in _get_next_batch
    generator_output = next(generator)
  File "c:\users\yemq3\appdata\local\programs\python\python37\lib\site-

KeyboardInterrupt: 